In [1]:
%pip install aiohttp

In [55]:
import aiohttp
import asyncio
import xml.etree.ElementTree as ET
from datetime import datetime

async def fetch_quote(session, ticker):
    url = f'https://iss.moex.com/iss/engines/stock/markets/shares/boards/TQBR/securities/{ticker}.xml'
    async with session.get(url) as response:
        xml_data = await response.text()
        root = ET.fromstring(xml_data)
        for row in root.findall(".//row"):
            if 'LAST' in row.attrib:
                return float(row.attrib['LAST'])
        return None


async def get_moexbc(tickers_data):
    print("Начинаем расчет индекса MOEXBC")
    print(f"Количество компаний: {len(tickers_data)}")
    print("\n" + "="*140)
    print("| {:^6} | {:^10} | {:^10} | {:^10} | {:^10} | {:^10} | {:^10} | {:^10} | {:^10} | {:^10} | {:^10} |".format(
        "Тикер", "База кап", "База цена", "Вес", "Общ. кап", "Б инд", "Дел", "Тек. цена", "Тек. кап", "Н кап", "Индекс"))
    print("="*140)


    total_base = sum(data['base_cap'] for data in tickers_data.values())
    base_idx = 20038.52
    divider = base_idx / total_base

    async with aiohttp.ClientSession() as session:
        tasks = [fetch_quote(session, ticker) for ticker in tickers_data]
        prices = await asyncio.gather(*tasks)
        current_caps = []
        for ticker, price in zip(tickers_data, prices):
            data = tickers_data[ticker]
            data['last_price'] = price if price else 0.0
            data['current_cap'] = data['base_cap'] / data['base_price'] * data['last_price'] if data['base_price'] != 0 else 0
            data['weight'] = data['base_cap'] / total_base
            current_caps.append(data['current_cap'])

        total_current = round(sum(current_caps), 2)

        for i, (ticker, price) in enumerate(zip(tickers_data, prices)):
            data = tickers_data[ticker]
            total_base_display = round(total_base, 2) if i == 0 else ""
            base_idx_display = base_idx if i == 0 else ""
            total_current_display = total_current if i == 0 else ""
            divider_display = round(divider,2) if i == 0 else ""
            new_idx = (divider_display * total_current_display) if i == 0 else ""

            print("| {:^6} | {:>10.2f} | {:>10.2f} | {:>10.4f} | {:>10} | {:>10} | {:>10} | {:>10.2f} | {:>10.2f} | {:>10} | {:>10} |".format(
                ticker,
                round(data['base_cap'], 2),
                round(data['base_price'], 2),
                data['weight'],
                total_base_display,
                base_idx_display,
                divider_display,
                data['last_price'],
                data['current_cap'],
                total_current_display,
                new_idx
            ))

    print("="*140)

async def main():
    tickers_data = {
        'CHMF': {'base_cap': 110.595, 'base_price': 1435.00},
        'GAZP': {'base_cap': 643.752, 'base_price': 162.25},
        'GMKN': {'base_cap': 388.556, 'base_price': 163.96},
        'IRAO': {'base_cap': 89.209, 'base_price': 4.069},
        'LKOH': {'base_cap': 772.998, 'base_price': 6911.50},
        'MGNT': {'base_cap': 109.185, 'base_price': 7239.00},
        'NLMK': {'base_cap': 92.078, 'base_price': 182.90},
        'NVTK': {'base_cap': 189.247, 'base_price': 1484.00},
        'PLZL': {'base_cap': 130.218, 'base_price': 10875.00},
        'ROSN': {'base_cap': 139.966, 'base_price': 600.30},
        'RUAL': {'base_cap': 95.415, 'base_price': 34.89},
        'SBER': {'base_cap': 701.917, 'base_price': 276.00},
        'SNGS': {'base_cap': 171.632, 'base_price': 28.215},
        'TATN': {'base_cap': 321.656, 'base_price': 688.20},
        'YDEX': {'base_cap': 83.504, 'base_price': 2611.00}
    }

    print("\nЗапуск расчета индекса MOEXBC")
    print("="*140)

    start_time = datetime.now()
    await get_moexbc(tickers_data)
    duration = (datetime.now() - start_time).total_seconds()

    print(f"\nВремя выполнения: {duration:.2f} секунд")
    print("="*140)

if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()
    asyncio.run(main())


Запуск расчета индекса MOEXBC
Начинаем расчет индекса MOEXBC
Количество компаний: 15

| Тикер  |  База кап  | База цена  |    Вес     |  Общ. кап  |   Б инд    |    Дел     | Тек. цена  |  Тек. кап  |   Н кап    |   Индекс   |
|  CHMF  |     110.59 |    1435.00 |     0.0274 |    4039.93 |   20038.52 |       4.96 |    1105.00 |      85.16 |    3486.16 | 17291.3536 |
|  GAZP  |     643.75 |     162.25 |     0.1593 |            |            |            |     128.54 |     510.00 |            |            |
|  GMKN  |     388.56 |     163.96 |     0.0962 |            |            |            |     111.94 |     265.28 |            |            |
|  IRAO  |      89.21 |       4.07 |     0.0221 |            |            |            |       3.57 |      78.27 |            |            |
|  LKOH  |     773.00 |    6911.50 |     0.1913 |            |            |            |    6587.50 |     736.76 |            |            |
|  MGNT  |     109.19 |    7239.00 |     0.0270 |            |     